Проективное преобразование


In [0]:
from type import ObjVertex
from sys import float_info
from core import CustomImage, ObjModel, face_angle_cos, barisentrik_coordinates
from names import DEER, RABBIT


def draw_faces(obj: ObjModel, _img: CustomImage, scale: tuple[float, float]):
    z_buffer = [[float_info.max for _ in range(_img.width())] for _ in range(_img.height())]
    img_x, img_y = _img.width() / 2, _img.height() / 2
    ax, ay = scale

    def transform(_v: ObjVertex) -> ObjVertex:
        _x, _y, _z = _v
        return (ax * _x + img_x * _z) / _z, (ay * _y + img_y * _z) / _z, _z

    for k, f in enumerate(obj.faces()):
        cos = face_angle_cos(f)
        if cos > 0:
            continue

        val = round(255 * -cos)
        color = (val, val, val)

        v0, v1, v2 = f

        v0 = transform(v0)
        v1 = transform(v1)
        v2 = transform(v2)

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        x_min = round(max(min(x0, x1, x2), 0))
        y_min = round(max(min(y0, y1, y2), 0))
        x_max = round(min(max(x0, x1, x2), _img.width() - 1))
        y_max = round(min(max(y0, y1, y2), _img.height() - 1))

        for i in range(x_min, x_max + 1):
            for j in range(y_min, y_max + 1):
                v = (i, j)
                l0, l1, l2 = barisentrik_coordinates(v, v0, v1, v2)
                if l0 > 0 and l1 > 0 and l2 > 0:
                    z = l0 * z0 + l1 * z1 + l2 * z2
                    if z < z_buffer[i][j]:
                        _img.set(v, color)
                        z_buffer[i][j] = z


deer_obj = ObjModel(DEER)
rabbit_obj = ObjModel(RABBIT)

img = CustomImage(1500, 1500)
deer_obj.shift((0, -800, 1000))
draw_faces(deer_obj, img, (700, 700))
img.save('pictures/faces/deer_projection.png')

img = CustomImage(1000, 1000)
rabbit_obj.shift((0.005, -0.045, 15.0))
draw_faces(rabbit_obj, img, (100000, 100000))
img.save('pictures/faces/rabbit_projection.png')

Поворот модели

In [0]:
from type import ObjVertex
from sys import float_info
import math
from core import CustomImage, ObjModel, face_angle_cos, barisentrik_coordinates
from names import DEER, RABBIT


def draw_faces(obj: ObjModel, _img: CustomImage, scale: tuple[float, float]):
    z_buffer = [[float_info.max for _ in range(_img.width())] for _ in range(_img.height())]
    img_x, img_y = _img.width() / 2, _img.height() / 2
    ax, ay = scale

    def transform(_v: ObjVertex) -> ObjVertex:
        _x, _y, _z = _v
        return (ax * _x + img_x * _z) / _z, (ay * _y + img_y * _z) / _z, _z

    for k, f in enumerate(obj.faces()):
        cos = face_angle_cos(f)
        if cos > 0:
            continue

        val = round(255 * -cos)
        color = (val, val, val)

        v0, v1, v2 = f

        v0 = transform(v0)
        v1 = transform(v1)
        v2 = transform(v2)

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        x_min = round(max(min(x0, x1, x2), 0))
        y_min = round(max(min(y0, y1, y2), 0))
        x_max = round(min(max(x0, x1, x2), _img.width() - 1))
        y_max = round(min(max(y0, y1, y2), _img.height() - 1))

        for i in range(x_min, x_max + 1):
            for j in range(y_min, y_max + 1):
                v = (i, j)
                l0, l1, l2 = barisentrik_coordinates(v, v0, v1, v2)
                if l0 > 0 and l1 > 0 and l2 > 0:
                    z = l0 * z0 + l1 * z1 + l2 * z2
                    if z < z_buffer[i][j]:
                        _img.set(v, color)
                        z_buffer[i][j] = z


deer_obj = ObjModel(DEER)
rabbit_obj = ObjModel(RABBIT)

img = CustomImage(1500, 1500)
deer_obj.rotate((math.pi, math.pi / 4, 0))
deer_obj.shift((0, 700, 10000))
draw_faces(deer_obj, img, (7000, 7000))
img.save('pictures/faces/deer_angle.png')

img = CustomImage(1000, 1000)
rabbit_obj.rotate((math.pi, 0, 0))
rabbit_obj.shift((0.005, 0.045, 15.0))
draw_faces(rabbit_obj, img, (100000, 100000))
img.save('pictures/faces/rabbit_angle.png')